Extract mel-spectrogram from audio

In [13]:
from sklearn.model_selection import KFold
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense, Flatten
from keras.optimizers import Adam
from keras.metrics import Precision, Recall
import os
import numpy as np
from keras.preprocessing.image import load_img, img_to_array
from keras.applications.vgg16 import preprocess_input

Find graphic specs

In [14]:
def load_data(dir_path):
    X = []
    y = []
    labels = ['PD', 'HC']
    for label in labels:
        path = os.path.join(dir_path, label)
        class_num = labels.index(label)
        for img in os.listdir(path):
            try:
                img_arr = img_to_array(load_img(os.path.join(path, img), target_size=(640, 480)))  # Convert image to array
                X.append(preprocess_input(img_arr))  # Preprocess the image using VGG16's preprocess_input method
                y.append(class_num)
            except Exception as e:
                print(e)
    return np.array(X), np.array(y)

In [15]:
X, y = load_data('./plots/')

# Define the K-fold Cross Validator
kfold = KFold(n_splits=5, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
histories = []
scores_per_fold = []

cannot identify image file <_io.BytesIO object at 0x2e9038310>


train

In [16]:
# Load VGG16 model, without the top layer (classification layer)
for train, test in kfold.split(X, y):
   vgg_model = VGG16(weights="imagenet", include_top=False, input_shape=(640, 480, 3))

   # Freeze the layers of VGG16 model to retain their pre-trained weights
   for layer in vgg_model.layers:
      layer.trainable = False

   # Add custom layers
   x = Flatten()(vgg_model.output)
   x = Dense(1024, activation="relu")(x)
   predictions = Dense(1, activation="sigmoid")(x)

   # Define the new model
   model = Model(inputs=vgg_model.input, outputs=predictions)

   # Compile the model
   model.compile(optimizer=Adam(lr=0.0001), 
               loss='binary_crossentropy', 
               metrics=['accuracy', Precision(), Recall()])

   # Generate a print
   print('------------------------------------------------------------------------')
   print(f'Training for fold {fold_no} ...')

   # Fit data to model
   history = model.fit(X[train], y[train],
            batch_size=32,
            epochs=20,
            verbose=1)
   histories.append(history)

   # Generate generalization metrics
   scores = model.evaluate(X[test], y[test], verbose=0)
   scores_per_fold.append(scores)

   fold_no = fold_no + 1

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/20
2/2 [==============================] - 17s 3s/step - loss: 142.9741 - accuracy: 0.4500 - precision_5: 0.4000 - recall_5: 0.7500
Epoch 2/20
2/2 [==============================] - 16s 3s/step - loss: 334.8394 - accuracy: 0.6000 - precision_5: 0.0000e+00 - recall_5: 0.0000e+00
Epoch 3/20
2/2 [==============================] - 16s 3s/step - loss: 9.8176 - accuracy: 0.8000 - precision_5: 0.7222 - recall_5: 0.8125
Epoch 4/20
2/2 [==============================] - 16s 3s/step - loss: 37.3714 - accuracy: 0.5250 - precision_5: 0.4571 - recall_5: 1.0000
Epoch 5/20
2/2 [==============================] - 16s 3s/step - loss: 2.4495 - accuracy: 0.9000 - precision_5: 0.8000 - recall_5: 1.0000
Epoch 6/20
2/2 [==============================] - 17s 3s/step - loss: 0.1256 - accuracy: 0.9750 - precision_5: 1.0000 - recall_5: 0.9375
Epoch 7/20
2/2 [==============================] - 17s 3s/step - loss

------------------------------------------------------------------------
Training for fold 2 ...
Epoch 1/20
2/2 [==============================] - 18s 3s/step - loss: 102.6790 - accuracy: 0.5122 - precision_6: 0.4839 - recall_6: 0.7895
Epoch 2/20
2/2 [==============================] - 17s 3s/step - loss: 403.9609 - accuracy: 0.5366 - precision_6: 0.0000e+00 - recall_6: 0.0000e+00
Epoch 3/20
2/2 [==============================] - 16s 3s/step - loss: 41.9285 - accuracy: 0.8537 - precision_6: 0.7600 - recall_6: 1.0000
Epoch 4/20
2/2 [==============================] - 16s 3s/step - loss: 140.2924 - accuracy: 0.4878 - precision_6: 0.4750 - recall_6: 1.0000
Epoch 5/20
2/2 [==============================] - 16s 3s/step - loss: 8.0133 - accuracy: 0.8780 - precision_6: 0.7917 - recall_6: 1.0000
Epoch 6/20
2/2 [==============================] - 17s 3s/step - loss: 4.0807 - accuracy: 0.9024 - precision_6: 1.0000 - recall_6: 0.7895
Epoch 7/20
2/2 [==============================] - 17s 3s/step - lo

------------------------------------------------------------------------
Training for fold 3 ...
Epoch 1/20
2/2 [==============================] - 18s 3s/step - loss: 113.8857 - accuracy: 0.5122 - precision_7: 0.5000 - recall_7: 0.3000
Epoch 2/20
2/2 [==============================] - 17s 3s/step - loss: 232.6901 - accuracy: 0.4878 - precision_7: 0.4857 - recall_7: 0.8500
Epoch 3/20
2/2 [==============================] - 17s 3s/step - loss: 74.7527 - accuracy: 0.5366 - precision_7: 1.0000 - recall_7: 0.0500   
Epoch 4/20
2/2 [==============================] - 17s 3s/step - loss: 0.0000e+00 - accuracy: 1.0000 - precision_7: 1.0000 - recall_7: 1.0000
Epoch 5/20
2/2 [==============================] - 17s 3s/step - loss: 15.8601 - accuracy: 0.8293 - precision_7: 0.7407 - recall_7: 1.0000
Epoch 6/20
2/2 [==============================] - 17s 3s/step - loss: 18.3424 - accuracy: 0.8293 - precision_7: 0.7407 - recall_7: 1.0000
Epoch 7/20
2/2 [==============================] - 16s 3s/step - los

------------------------------------------------------------------------
Training for fold 4 ...
Epoch 1/20
2/2 [==============================] - 18s 3s/step - loss: 114.8880 - accuracy: 0.3659 - precision_8: 0.3200 - recall_8: 0.4706
Epoch 2/20
2/2 [==============================] - 17s 3s/step - loss: 148.9655 - accuracy: 0.6829 - precision_8: 1.0000 - recall_8: 0.2353    
Epoch 3/20
2/2 [==============================] - 18s 4s/step - loss: 52.3230 - accuracy: 0.5366 - precision_8: 0.4722 - recall_8: 1.0000
Epoch 4/20
2/2 [==============================] - 18s 3s/step - loss: 4.2329 - accuracy: 0.9024 - precision_8: 0.8095 - recall_8: 1.0000
Epoch 5/20
2/2 [==============================] - 16s 3s/step - loss: 3.0551 - accuracy: 0.9512 - precision_8: 1.0000 - recall_8: 0.8824
Epoch 6/20
2/2 [==============================] - 17s 3s/step - loss: 17.3333 - accuracy: 0.8537 - precision_8: 1.0000 - recall_8: 0.6471
Epoch 7/20
2/2 [==============================] - 17s 3s/step - loss: 1

------------------------------------------------------------------------
Training for fold 5 ...
Epoch 1/20
2/2 [==============================] - 18s 3s/step - loss: 123.1206 - accuracy: 0.5854 - precision_9: 0.5556 - recall_9: 0.7500
Epoch 2/20
2/2 [==============================] - 17s 3s/step - loss: 204.2229 - accuracy: 0.6098 - precision_9: 1.0000 - recall_9: 0.2000    
Epoch 3/20
2/2 [==============================] - 17s 3s/step - loss: 98.3699 - accuracy: 0.5366 - precision_9: 0.5128 - recall_9: 1.0000
Epoch 4/20
2/2 [==============================] - 17s 3s/step - loss: 18.6873 - accuracy: 0.7805 - precision_9: 0.6897 - recall_9: 1.0000
Epoch 5/20
2/2 [==============================] - 17s 3s/step - loss: 6.4203 - accuracy: 0.9024 - precision_9: 1.0000 - recall_9: 0.8000
Epoch 6/20
2/2 [==============================] - 18s 4s/step - loss: 48.8561 - accuracy: 0.6098 - precision_9: 1.0000 - recall_9: 0.2000
Epoch 7/20
2/2 [==============================] - 17s 4s/step - loss: 

In [17]:
# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(scores_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {scores_per_fold[i][0]} - Accuracy: {scores_per_fold[i][1]*100}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean([scores[1] for scores in scores_per_fold])*100} (+- {np.std([scores[1] for scores in scores_per_fold])})')
print(f'> Loss: {np.mean([scores[0] for scores in scores_per_fold])}')
print('------------------------------------------------------------------------')

------------------------------------------------------------------------
Score per fold
------------------------------------------------------------------------
> Fold 1 - Loss: 3.7864780755292315e-12 - Accuracy: 100.0%
------------------------------------------------------------------------
> Fold 2 - Loss: 1.7263095397623829e-09 - Accuracy: 100.0%
------------------------------------------------------------------------
> Fold 3 - Loss: 5.12715482711792 - Accuracy: 89.99999761581421%
------------------------------------------------------------------------
> Fold 4 - Loss: 38.119117736816406 - Accuracy: 80.0000011920929%
------------------------------------------------------------------------
> Fold 5 - Loss: 5.3505539894104 - Accuracy: 80.0000011920929%
------------------------------------------------------------------------
Average scores for all folds:
> Accuracy: 90.0 (+- 0.08944271376879089)
> Loss: 9.719365311014965
----------------------------------------------------------------

In [2]:
print(scores_per_fold)


NameError: name 'scores_per_fold' is not defined